In [4]:
import openfoodfacts
import pandas as pd

    # List available attributes and methods in the library
    
    print(dir(openfoodfacts))


['API', 'APIConfig', 'APIVersion', 'Country', 'DatasetType', 'Environment', 'Facet', 'Flavor', 'Lang', 'OCRResult', 'ProductDataset', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'api', 'dataset', 'get_dataset', 'ocr', 'types', 'utils']


In [5]:
import requests
import pandas as pd

# Initialize an empty list to store barcodes
barcodes = []

# Parameters for pagination
page_number = 1
page_size = 100  # Number of products per request

while True:
    params = {
        'search_terms': '',  # No search terms to get all products
        'page_size': page_size,
        'page': page_number,
        'json': 1  # Request results in JSON format
    }
    
    response = requests.get("https://world.openfoodfacts.org/cgi/search.pl", params=params)
    
    if response.status_code != 200:
        print(f"Failed to fetch page {page_number}. Status code: {response.status_code}")
        break  # Stop if request fails
    
    products = response.json().get('products', [])
    
    if not products:
        print("No more products found. Stopping collection.")
        break  # Stop if no more products
    
    for product in products:
        barcode = product.get('code')
        if barcode:
            barcodes.append(barcode)
    
    print(f"Page {page_number} processed, {len(barcodes)} barcodes collected so far.")
    page_number += 1

# Convert the barcodes to a Pandas DataFrame
df_barcodes = pd.DataFrame(barcodes, columns=['Barcode'])

# Display the first few rows of the DataFrame
df_barcodes.head()


Page 1 processed, 100 barcodes collected so far.
Page 2 processed, 200 barcodes collected so far.
Page 3 processed, 300 barcodes collected so far.
Page 4 processed, 400 barcodes collected so far.
Page 5 processed, 500 barcodes collected so far.
Page 6 processed, 600 barcodes collected so far.
Page 7 processed, 700 barcodes collected so far.
Page 8 processed, 800 barcodes collected so far.
Page 9 processed, 900 barcodes collected so far.
Page 10 processed, 1000 barcodes collected so far.
Page 11 processed, 1100 barcodes collected so far.
Page 12 processed, 1200 barcodes collected so far.
Page 13 processed, 1300 barcodes collected so far.
Page 14 processed, 1400 barcodes collected so far.
Page 15 processed, 1500 barcodes collected so far.
Page 16 processed, 1600 barcodes collected so far.
Page 17 processed, 1700 barcodes collected so far.
Page 18 processed, 1800 barcodes collected so far.
Page 19 processed, 1900 barcodes collected so far.
Page 20 processed, 2000 barcodes collected so far

,Barcode
0,3274080005003
1,7622210449283
2,3017620425035
3,3175680011480
4,5449000214911


In [8]:
df_barcodes.shape[0]

5100

In [9]:
df_barcodes.to_csv('barcodes_v2.csv', index=False)

In [ ]:
# DataFrame to store food items and ingredients
food_data = []

for barcode in df_barcodes['Barcode']:
    response = requests.get(f"https://world.openfoodfacts.org/api/v0/product/{barcode}.json")
    
    if response.status_code == 200:
        product_data = response.json()
        
        if product_data['status'] == 1:
            product = product_data['product']
            food_item = product.get('product_name', 'No name')
            ingredients = product.get('ingredients_text', 'No ingredients available')
            food_data.append([food_item, ingredients])
    
# Convert the food items and ingredients into a DataFrame
df_food = pd.DataFrame(food_data, columns=['Food Item', 'Ingredients'])

# Display the first few rows of the DataFrame
df_food.head()


In [ ]:
df_food.to_csv('sf_foods_v2.csv', index=False)